Before running this notebook setup the workspace and generate the results using:
```bash
julia src/setup.jl
parallel "julia src/run.jl {1} {2}" ::: cp hs idr qrm qra iqra lqra ::: {1..24}
```

In [ ]:
using PostForecasts, Plots, Statistics

In [ ]:
results = Dict()
models = [:hs, :cp, :idr, :qra, :qrm, :lqra, :iqra]
labels = ["HS", "CP", "IDR", "QRA", "QRM", "LQRA", "iQRA"]
for model in models
    results[model] = Vector{QuantForecasts{Float64, Int64}}(undef, 24)
    for h in 1:24
        results[model][h] = loadquantf("outputs/$(model)$(h)")
    end
end

In [ ]:
# CRPS per year
for year in [2020, 2021, 2022, 2023, 2024]
    println("$(year)")
    println("*"^20)
    for model in models
        println(round(mean([0.5*crps(results[model][h](year*10_000+0101, year*10_000+1231)) for h in 1:24]), digits=3))
    end
    println("*"^20)
end

In [ ]:
# PIPS
for alpha in [2, 4, 10, 20]
    upper = Int(100-alpha/2)
    lower = Int(alpha/2)
    println("$(alpha)%")
    println("*"^20)
    for model in models
        println(round(mean([mean(pinball(results[model][h])[[lower, upper]]) for h in 1:24]), digits=3))
    end
    println("*"^20)
end

In [ ]:
plot(layout=(2, 2), size = (800, 600))
PIs = [2, 4, 10, 20]
for subplot in 1:4
    upper = Int(100-(PIs[subplot]/2))
    lower = Int(PIs[subplot]/2)
    for (m, model) in pairs(models)
        cov = mean([coverage(results[model][h](20200101, 20241231))[[lower, upper]] for h in 1:24])
        y = 100*(cov[2] - cov[1]) - (upper - lower)
        x1 = 100*cov[1]
        x2 = 100*(1-cov[2])
        x = x1 - x2
        label = labels[m]
        plot!(legend = (subplot==4 ? :right : nothing))
        plot!(xlims = (-2, 2))
        plot!(ylims = (-12, 0))
        plot!(title = "$(100-PIs[subplot])% PI", subplot=subplot)
        plot!([x], [y], framestyle=:zerolines, label=label, st=:scatter, markershape=:circle, markersize=6, subplot=subplot)
    end
end
plot!(xlabel = "TB (%)")
plot!(ylabel = "ACE (%)")